<a href="https://colab.research.google.com/github/thanh-abaii/OpenDeepResearcher/blob/main/open_deep_researcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nest_asyncio
!pip install tavily-python
!pip install groq
import nest_asyncio
nest_asyncio.apply()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.1 MB/s eta 0:00:00


In [4]:
import asyncio
import aiohttp
import os
from tavily import TavilyClient
from google.colab import userdata

# =======================
# Configuration Constants
# =======================
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')  # Replace with your OpenRouter API key
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')  # Replace with your Tavily API key

OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
DEFAULT_MODEL = "deepseek/deepseek-r1-distill-qwen-32b"

# Instantiate Tavily Client (synchronous, used via asyncio.to_thread)
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [6]:
response = tavily_client.search("Who is Leo Messi?")

print(response)

{'query': 'Who is Leo Messi?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Career of Lionel Messi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Career_of_Lionel_Messi', 'content': "Lionel Messi is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.His individual achievements include eight Ballon d'Or awards, the most for any footballer. Having won 45 team trophies, [note 1] he is the most decorated player in the history of professional football. [11]", 'score': 0.8405867, 'raw_content': None}, {'title': 'Lionel Messi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'content': 'He scored twice in the last group match, a 3–2 victory over Nigeria, his second goal coming from a free kick, as they finished first in their group.[423] Messi assisted a late goal in extra time to ensure a 1–0 win against Switzerland in the round of 16,

In [12]:
import asyncio
import aiohttp
from tavily import TavilyClient
from google.colab import userdata

# =======================
# Configuration Constants
# =======================
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')  # Replace with your OpenRouter API key
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')            # Replace with your Tavily API key

OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
DEFAULT_MODEL = "deepseek/deepseek-r1-distill-qwen-32b"

# Instantiate Tavily Client (synchronous, used via asyncio.to_thread)
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# ============================
# Asynchronous Helper Functions
# ============================

async def call_openrouter_async(session, messages, model=DEFAULT_MODEL):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "X-Title": "OpenDeepResearcher, by Matt Shumer",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": messages
    }
    try:
        async with session.post(OPENROUTER_URL, headers=headers, json=payload) as resp:
            if resp.status == 200:
                result = await resp.json()
                try:
                    return result['choices'][0]['message']['content']
                except (KeyError, IndexError):
                    print("Unexpected OpenRouter response structure:", result)
                    return None
            else:
                text = await resp.text()
                print(f"OpenRouter API error: {resp.status} - {text}")
                return None
    except Exception as e:
        print("Error calling OpenRouter:", e)
        return None

async def generate_search_queries_async(session, user_query):
    prompt = (
        "You are an expert research assistant. Given the user's query, generate up to four distinct, "
        "precise search queries that would help gather comprehensive information on the topic. "
        "Return only a Python list of strings, for example: ['query1', 'query2', 'query3']."
    )
    messages = [
        {"role": "system", "content": "You are a helpful and precise research assistant."},
        {"role": "user", "content": f"User Query: {user_query}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    if response:
        try:
            search_queries = eval(response)
            if isinstance(search_queries, list):
                return search_queries
            else:
                print("LLM did not return a list. Response:", response)
                return []
        except Exception as e:
            print("Error parsing search queries:", e, "\nResponse:", response)
            return []
    return []

async def perform_search_async(session, query):
    """
    Asynchronously perform a search using Tavily Search for the given query.
    """
    try:
        response = await asyncio.to_thread(tavily_client.search, query)
        links = []
        # Tavily API might return results in a key "results" (or "organic_results")
        if "organic_results" in response:
            links = [item.get("link") for item in response["organic_results"] if "link" in item]
        elif "results" in response:
            links = [item.get("url") for item in response["results"] if "url" in item]
        else:
            print("No recognizable results in Tavily search response for query:", query)
        return links
    except Exception as e:
        print("Error performing Tavily search:", e)
        return []

async def extract_webpage_text_async(session, url):
    """
    Asynchronously extract the raw text content of a webpage using Tavily's extraction API.
    """
    try:
        # Tavily's extract expects a list of URLs.
        response = await asyncio.to_thread(tavily_client.extract, urls=[url], include_images=False)
        if "results" in response and len(response["results"]) > 0:
            result = response["results"][0]
            return result.get("raw_content", "")
        else:
            print(f"No extraction result for URL: {url}")
            return ""
    except Exception as e:
        print("Error extracting webpage text with Tavily:", e)
        return ""

async def is_page_useful_async(session, user_query, page_text):
    prompt = (
        "You are a critical research evaluator. Given the user's query and the content of a webpage, "
        "determine if the webpage contains information relevant and useful for addressing the query. "
        "Respond with exactly one word: 'Yes' if the page is useful, or 'No' if it is not. Do not include any extra text."
    )
    messages = [
        {"role": "system", "content": "You are a strict and concise evaluator of research relevance."},
        {"role": "user", "content": f"User Query: {user_query}\n\nWebpage Content (first 20000 characters):\n{page_text[:20000]}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    if response:
        answer = response.strip()
        if answer in ["Yes", "No"]:
            return answer
        else:
            if "Yes" in answer:
                return "Yes"
            elif "No" in answer:
                return "No"
    return "No"

async def extract_relevant_context_async(session, user_query, search_query, page_text):
    prompt = (
        "You are an expert information extractor. Given the user's query, the search query that led to this page, "
        "and the webpage content, extract all pieces of information that are relevant to answering the user's query. "
        "Return only the relevant context as plain text without commentary."
    )
    messages = [
        {"role": "system", "content": "You are an expert in extracting and summarizing relevant information."},
        {"role": "user", "content": f"User Query: {user_query}\nSearch Query: {search_query}\n\nWebpage Content (first 20000 characters):\n{page_text[:20000]}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    if response:
        return response.strip()
    return ""

async def get_new_search_queries_async(session, user_query, previous_search_queries, all_contexts):
    context_combined = "\n".join(all_contexts) if all_contexts else ""
    prompt = (
        "You are an analytical research assistant. Based on the original query, the search queries performed so far, "
        "and the extracted contexts from webpages (if any), determine if further research is needed. "
        "If further research is needed, provide up to four new search queries as a Python list (for example, "
        "['new query1', 'new query2']). If you believe no further research is needed, respond with exactly <done>."
        "\nOutput only a Python list or the token <done> without any additional text."
    )
    messages = [
        {"role": "system", "content": "You are a systematic research planner."},
        {"role": "user", "content": f"User Query: {user_query}\nPrevious Search Queries: {previous_search_queries}\n\nExtracted Relevant Contexts:\n{context_combined}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    if response:
        cleaned = response.strip()
        if cleaned == "<done>":
            return "<done>"
        try:
            new_queries = eval(cleaned)
            if isinstance(new_queries, list):
                return new_queries
            else:
                print("LLM did not return a list for new search queries. Response:", response)
                return []
        except Exception as e:
            print("Error parsing new search queries:", e, "\nResponse:", response)
            return []
    return []

async def generate_final_report_async(session, user_query, all_contexts):
    if not all_contexts:
        return f"No relevant content was found for the query: {user_query}."
    context_combined = "\n".join(all_contexts)
    prompt = (
        "You are an expert researcher and report writer. Based on the gathered contexts below and the original query, "
        "write a comprehensive, well-structured, and detailed report that addresses the query thoroughly. "
        "Include all relevant insights and conclusions without extraneous commentary."
    )
    messages = [
        {"role": "system", "content": "You are a skilled report writer."},
        {"role": "user", "content": f"User Query: {user_query}\n\nGathered Relevant Contexts:\n{context_combined}\n\n{prompt}"}
    ]
    report = await call_openrouter_async(session, messages)
    return report

async def process_link(session, link, user_query, search_query):
    print(f"Fetching and extracting content from: {link}")
    page_text = await extract_webpage_text_async(session, link)
    if not page_text:
        return None
    usefulness = await is_page_useful_async(session, user_query, page_text)
    print(f"Page usefulness for {link}: {usefulness}")
    if usefulness == "Yes":
        context = await extract_relevant_context_async(session, user_query, search_query, page_text)
        if context:
            print(f"Extracted context from {link} (first 200 chars): {context[:200]}")
            return context
    return None

# =========================
# Main Asynchronous Routine
# =========================

async def async_main():
    user_query = input("Enter your research query/topic: ").strip()
    iter_limit_input = input("Enter maximum number of iterations (default 10): ").strip()
    iteration_limit = int(iter_limit_input) if iter_limit_input.isdigit() else 10

    aggregated_contexts = []    # All useful contexts from every iteration
    all_search_queries = []     # Every search query used across iterations
    iteration = 0

    async with aiohttp.ClientSession() as session:
        # ----- INITIAL SEARCH QUERIES -----
        new_search_queries = await generate_search_queries_async(session, user_query)
        if not new_search_queries:
            print("No search queries were generated by the LLM. Exiting.")
            return
        all_search_queries.extend(new_search_queries)

        # ----- ITERATIVE RESEARCH LOOP -----
        while iteration < iteration_limit:
            print(f"\n=== Iteration {iteration + 1} ===")
            iteration_contexts = []

            # For each search query, perform Tavily searches concurrently.
            search_tasks = [perform_search_async(session, query) for query in new_search_queries]
            search_results = await asyncio.gather(*search_tasks)

            # Aggregate all unique links from all search queries of this iteration.
            unique_links = {}
            for idx, links in enumerate(search_results):
                query = new_search_queries[idx]
                for link in links:
                    if link and link not in unique_links:
                        unique_links[link] = query

            if unique_links:
                print(f"Aggregated {len(unique_links)} unique links from this iteration:")
                for url, query in unique_links.items():
                    print(f"  - URL: {url} (from query: {query})")
            else:
                print("No links found for the current search queries.")

            # Process each link concurrently.
            if unique_links:
                link_tasks = [
                    process_link(session, link, user_query, unique_links[link])
                    for link in unique_links
                ]
                link_results = await asyncio.gather(*link_tasks)
                for res in link_results:
                    if res:
                        iteration_contexts.append(res)
            else:
                print("Skipping link processing due to lack of results.")

            if iteration_contexts:
                aggregated_contexts.extend(iteration_contexts)
            else:
                print("No useful contexts were found in this iteration.")

            # ----- ASK THE LLM IF MORE SEARCHES ARE NEEDED -----
            new_search_queries = await get_new_search_queries_async(session, user_query, all_search_queries, aggregated_contexts)
            if new_search_queries == "<done>":
                print("LLM indicated that no further research is needed.")
                break
            elif new_search_queries:
                print("LLM provided new search queries:", new_search_queries)
                all_search_queries.extend(new_search_queries)
            else:
                print("LLM did not provide any new search queries. Ending the loop.")
                break

            iteration += 1

        # ----- FINAL REPORT -----
        print("\nGenerating final report...")
        final_report = await generate_final_report_async(session, user_query, aggregated_contexts)
        print("\n==== FINAL REPORT ====\n")
        print(final_report)

def main():
    asyncio.run(async_main())

if __name__ == "__main__":
    main()


Enter your research query/topic: DeepSeek-R1
Enter maximum number of iterations (default 10): 3
Unexpected OpenRouter response structure: {'error': {'message': 'More credits are required to run this request. 117877 token capacity required, 101717 available. To increase, visit https://openrouter.ai/credits and upgrade to a paid account', 'code': 402}}
No search queries were generated by the LLM. Exiting.


## Phiên bản cải thiện

In [11]:
import asyncio
import aiohttp
from tavily import TavilyClient
from google.colab import userdata

# =======================
# Configuration Constants
# =======================
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
DEFAULT_MODEL = "deepseek/deepseek-r1-distill-qwen-32b"

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# ============================
# Asynchronous Helper Functions
# ============================

async def call_openrouter_async(session, messages, model=DEFAULT_MODEL):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "X-Title": "OpenDeepResearcher, by Matt Shumer",
        "Content-Type": "application/json"
    }
    payload = {"model": model, "messages": messages}

    try:
        async with session.post(OPENROUTER_URL, headers=headers, json=payload) as resp:
            if resp.status == 200:
                result = await resp.json()
                return result.get('choices', [{}])[0].get('message', {}).get('content', None)
            else:
                print(f"OpenRouter API error: {resp.status}")
                return None
    except Exception as e:
        print("Error calling OpenRouter:", e)
        return None

async def generate_search_queries_async(session, user_query):
    """
    Generate new search queries based on the user's research topic.
    """
    prompt = (
        "You are a research assistant. Generate up to four precise search queries "
        "that provide comprehensive information on the topic. Return a Python list, e.g.: ['query1', 'query2']."
    )
    messages = [
        {"role": "system", "content": "You are a precise research assistant."},
        {"role": "user", "content": f"User Query: {user_query}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)

    if response:
        # Loại bỏ dấu ```python
        cleaned_response = response.replace("```python", "").replace("```", "").strip()

        # Kiểm tra nếu phản hồi có dạng danh sách hợp lệ trước khi eval()
        if cleaned_response.startswith("[") and cleaned_response.endswith("]"):
            try:
                search_queries = eval(cleaned_response)
                if isinstance(search_queries, list):
                    return search_queries
            except Exception as e:
                print(f"Error parsing search queries: {e}\nResponse: {cleaned_response}")

        print(f"Invalid search query format: {cleaned_response}")

    return []


async def perform_search_async(session, query):
    """
    Perform a Tavily search with optimized parameters.
    """
    try:
        response = await asyncio.to_thread(
            tavily_client.search,
            query=query,
            max_results=10,  # Tăng số lượng kết quả để có nhiều dữ liệu hơn
            include_raw_content=True,  # Lấy nội dung văn bản thô từ trang web
            search_depth="advanced"  # Tìm kiếm chuyên sâu hơn
        )

        return [item.get("url") for item in response.get("results", []) if "url" in item]
    except Exception as e:
        print("Error performing Tavily search:", e)
        return []


async def extract_webpage_text_async(session, url):
    """
    Asynchronously extract the raw text content of a webpage using Tavily's extraction API.
    """
    try:
        response = await asyncio.to_thread(tavily_client.extract, urls=[url], include_images=False)

        if "results" in response and response["results"]:
            return response["results"][0].get("raw_content", "").strip()

        print(f"⚠️ No extraction result for URL: {url} - Response: {response}")
        return ""
    except Exception as e:
        print(f"❌ Error extracting webpage text with Tavily for {url}: {e}")
        return ""



async def is_page_useful_async(session, user_query, page_text):
    prompt = "Is the webpage useful for answering the query? Respond with exactly 'Yes' or 'No'."
    messages = [
        {"role": "system", "content": "You are a research evaluator."},
        {"role": "user", "content": f"User Query: {user_query}\nWebpage Content:\n{page_text[:20000]}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    return response.strip() if response in ["Yes", "No"] else "No"

async def extract_relevant_context_async(session, user_query, search_query, page_text):
    prompt = "Extract only relevant information from the webpage content for answering the user's query."
    messages = [
        {"role": "system", "content": "You are an expert information extractor."},
        {"role": "user", "content": f"User Query: {user_query}\nSearch Query: {search_query}\nWebpage Content:\n{page_text[:20000]}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    return response.strip() if response else ""

async def process_link(session, link, user_query, search_query):
    print(f"🔍 Fetching and extracting content from: {link}")

    # Thử trích xuất 2 lần nếu lần đầu thất bại
    for attempt in range(2):
        page_text = await extract_webpage_text_async(session, link)
        if page_text:
            break
        print(f"⚠️ Retry extracting content from {link} (Attempt {attempt + 1}/2)")

    if not page_text:
        print(f"❌ Failed to extract content from {link} after 2 attempts.")
        return None

    usefulness = await is_page_useful_async(session, user_query, page_text)
    if usefulness == "Yes":
        return await extract_relevant_context_async(session, user_query, search_query, page_text)

    return None


async def generate_final_report_async(session, user_query, all_contexts):
    if not all_contexts:
        return f"No relevant content was found for: {user_query}."

    context_combined = "\n".join(all_contexts)
    prompt = "Write a detailed and well-structured report based on the gathered contexts and the original query."
    messages = [
        {"role": "system", "content": "You are a skilled report writer."},
        {"role": "user", "content": f"User Query: {user_query}\nGathered Contexts:\n{context_combined}\n\n{prompt}"}
    ]
    return await call_openrouter_async(session, messages)

# =========================
# Main Asynchronous Routine
# =========================

async def async_main():
    user_query = input("Enter your research query/topic: ").strip()
    iter_limit_input = input("Enter max iterations (default 10): ").strip()
    iteration_limit = int(iter_limit_input) if iter_limit_input.isdigit() else 10

    aggregated_contexts = []
    all_search_queries = []
    iteration = 0

    async with aiohttp.ClientSession() as session:
        new_search_queries = await generate_search_queries_async(session, user_query)
        if not new_search_queries:
            print("❌ No search queries generated. Exiting.")
            return
        all_search_queries.extend(new_search_queries)

        while iteration < iteration_limit:
            print(f"\n=== Iteration {iteration + 1} ===")
            search_tasks = [perform_search_async(session, query) for query in new_search_queries]
            search_results = await asyncio.gather(*search_tasks)

            unique_links = {}
            for idx, links in enumerate(search_results):
                query = new_search_queries[idx]
                for link in links:
                    if link and link not in unique_links:
                        unique_links[link] = query

            if not unique_links:
                print("⚠️ No links found. Trying again in next iteration...")
                iteration += 1
                continue

            print(f"✅ Aggregated {len(unique_links)} unique links from this iteration.")
            link_tasks = [process_link(session, link, user_query, unique_links[link]) for link in unique_links]
            link_results = await asyncio.gather(*link_tasks)

            # Chỉ lấy các context hợp lệ
            iteration_contexts = [res for res in link_results if res]
            aggregated_contexts.extend(iteration_contexts)

            # Nếu không có kết quả hữu ích, đừng dừng luôn, thử tiếp
            if not iteration_contexts:
                print("⚠️ No useful content extracted, continuing to next iteration...")
                iteration += 1
                continue

            new_search_queries = await generate_search_queries_async(session, user_query)
            if not new_search_queries:
                print("🚫 No new queries. Ending loop.")
                break
            all_search_queries.extend(new_search_queries)
            iteration += 1

        # Nếu không có dữ liệu, báo cáo điều đó thay vì "None"
        print("\n📝 Generating final report...")
        if not aggregated_contexts:
            print("❌ No relevant content was extracted. Report cannot be generated.")
            return

        final_report = await generate_final_report_async(session, user_query, aggregated_contexts)
        print("\n==== FINAL REPORT ====\n")
        print(final_report)


def main():
    asyncio.run(async_main())

if __name__ == "__main__":
    main()


Enter your research query/topic: DeepSeek-R1
Enter max iterations (default 10): 3
❌ No search queries generated. Exiting.


## Dùng Groq và llama-3.3-70b-versatile model

In [23]:
import os
import asyncio
import aiohttp
from tavily import TavilyClient
from groq import Groq
from google.colab import userdata

# =======================
# Configuration Constants
# =======================
GROQ_API_KEY = userdata.get("GROQ_API_KEY")  # API key Groq
TAVILY_API_KEY = userdata.get("TAVILY_API_KEY")  # API key Tavily

DEFAULT_MODEL = "mixtral-8x7b-32768"

# Giới hạn số lượng trang được xử lý mỗi vòng lặp để tránh quá tải token
MAX_LINKS_PER_ITERATION = 10
MAX_TOKENS_INPUT = 4000  # Giới hạn tokens để tránh vượt quá quota

# Khởi tạo API client
groq_client = Groq(api_key=GROQ_API_KEY)
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# ============================
# Asynchronous Helper Functions
# ============================

async def call_groq_async(messages, model=DEFAULT_MODEL):
    """
    Gửi yêu cầu đến Groq API để xử lý hội thoại.
    """
    try:
        # Giới hạn số lượng tokens bằng cách cắt nội dung của các tin nhắn đầu vào
        for msg in messages:
            if "content" in msg:
                msg["content"] = msg["content"][:MAX_TOKENS_INPUT]

        chat_completion = await asyncio.to_thread(
            groq_client.chat.completions.create,
            messages=messages,
            model=model,
            temperature=0.7,
            max_completion_tokens=500  # Giới hạn số tokens trong phản hồi
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ Groq API Error: {e}")
        return None

async def generate_search_queries_async(user_query):
    """
    Tạo danh sách các truy vấn tìm kiếm dựa trên truy vấn của người dùng.
    """
    prompt = (
        "Generate up to four precise search queries as a Python list, e.g., ['query1', 'query2', 'query3']."
    )
    messages = [
        {"role": "system", "content": "You are a research assistant."},
        {"role": "user", "content": f"User Query: {user_query}\n\n{prompt}"}
    ]
    response = await call_groq_async(messages)

    if response:
        try:
            search_queries = eval(response)
            if isinstance(search_queries, list):
                return search_queries
        except Exception as e:
            print(f"Error parsing search queries: {e}\nResponse: {response}")

    print(f"⚠️ Invalid search query format: {response}")
    return []

async def perform_search_async(query):
    """
    Thực hiện tìm kiếm bằng Tavily API.
    """
    try:
        response = await asyncio.to_thread(
            tavily_client.search,
            query=query,
            max_results=5,  # Giảm số lượng kết quả để tránh quá tải
            include_raw_content=True
        )
        return [item.get("url") for item in response.get("results", []) if "url" in item]
    except Exception as e:
        print("❌ Error performing Tavily search:", e)
        return []

async def extract_webpage_text_async(url):
    """
    Trích xuất nội dung văn bản từ một trang web bằng Tavily API.
    """
    try:
        response = await asyncio.to_thread(tavily_client.extract, urls=[url], include_images=False)
        if "results" in response and response["results"]:
            return response["results"][0].get("raw_content", "")[:MAX_TOKENS_INPUT]  # Cắt nội dung
        print(f"⚠️ No extraction result for URL: {url}")
        return ""
    except Exception as e:
        print(f"❌ Error extracting webpage text with Tavily for {url}: {e}")
        return ""

async def is_page_useful_async(user_query, page_text):
    """
    Xác định xem nội dung của một trang có liên quan đến truy vấn người dùng hay không.
    """
    prompt = "Is this webpage useful for answering the query? Respond 'Yes' or 'No'."
    messages = [
        {"role": "system", "content": "You are an evaluator."},
        {"role": "user", "content": f"User Query: {user_query}\nWebpage Content:\n{page_text[:MAX_TOKENS_INPUT]}\n\n{prompt}"}
    ]
    response = await call_groq_async(messages)
    return response.strip() if response in ["Yes", "No"] else "No"

async def extract_relevant_context_async(user_query, search_query, page_text):
    """
    Trích xuất nội dung liên quan từ trang web.
    """
    prompt = "Extract only relevant information from the webpage content."
    messages = [
        {"role": "system", "content": "You are an expert extractor."},
        {"role": "user", "content": f"User Query: {user_query}\nSearch Query: {search_query}\nWebpage Content:\n{page_text[:MAX_TOKENS_INPUT]}\n\n{prompt}"}
    ]
    response = await call_groq_async(messages)
    return response.strip() if response else ""

async def generate_final_report_async(user_query, all_contexts):
    """
    Tạo báo cáo cuối cùng từ tất cả các nội dung đã thu thập.
    """
    if not all_contexts:
        return f"No relevant content found for: {user_query}."

    context_combined = "\n".join(all_contexts)
    prompt = "Write a detailed report based on the gathered contexts and the query."
    messages = [
        {"role": "system", "content": "You are a report writer."},
        {"role": "user", "content": f"User Query: {user_query}\nGathered Contexts:\n{context_combined[:MAX_TOKENS_INPUT]}\n\n{prompt}"}
    ]
    return await call_groq_async(messages)

async def process_link(link, user_query, search_query):
    """
    Xử lý từng link: trích xuất nội dung, đánh giá và lọc thông tin.
    """
    print(f"🔍 Fetching and extracting content from: {link}")
    page_text = await extract_webpage_text_async(link)
    if not page_text:
        return None

    usefulness = await is_page_useful_async(user_query, page_text)
    if usefulness == "Yes":
        return await extract_relevant_context_async(user_query, search_query, page_text)
    return None

# =========================
# Main Asynchronous Routine
# =========================

async def async_main():
    user_query = input("Enter your research query/topic: ").strip()
    iteration_limit = 2  # Giảm số vòng lặp để tránh quá tải token

    aggregated_contexts = []
    all_search_queries = await generate_search_queries_async(user_query)
    if not all_search_queries:
        print("❌ No search queries generated. Exiting.")
        return

    for iteration in range(iteration_limit):
        print(f"\n=== Iteration {iteration + 1} ===")
        search_results = await asyncio.gather(*[perform_search_async(query) for query in all_search_queries])

        unique_links = set()
        for links in search_results:
            unique_links.update(links)

        # Giới hạn số link được xử lý để tránh vượt quota
        limited_links = list(unique_links)[:MAX_LINKS_PER_ITERATION]
        link_results = await asyncio.gather(*[process_link(link, user_query, all_search_queries[0]) for link in limited_links])

        aggregated_contexts.extend(filter(None, link_results))

    print("\nGenerating final report...")
    final_report = await generate_final_report_async(user_query, aggregated_contexts)
    print("\n==== FINAL REPORT ====\n", final_report)

def main():
    asyncio.run(async_main())

if __name__ == "__main__":
    main()


Enter your research query/topic: deepseek R1

=== Iteration 1 ===
🔍 Fetching and extracting content from: https://www.mining.com/graph-mining-vs-ai-vs-deepseek/
🔍 Fetching and extracting content from: https://www.linkedin.com/pulse/deepseek-vs-chatgpt-redefining-ais-battlefront-jason-ledbetter-dtxve/
🔍 Fetching and extracting content from: https://www.techtarget.com/WhatIs/feature/DeepSeek-explained-Everything-you-need-to-know
🔍 Fetching and extracting content from: https://www.reuters.com/technology/artificial-intelligence/what-is-deepseek-why-is-it-disrupting-ai-sector-2025-01-27/
🔍 Fetching and extracting content from: https://www.getguru.com/reference/deepseek
🔍 Fetching and extracting content from: https://www.analyticsvidhya.com/blog/2025/01/ai-application-with-deepseek-v3/
🔍 Fetching and extracting content from: https://tldv.io/blog/what-is-deepseek/
🔍 Fetching and extracting content from: https://www.geeky-gadgets.com/best-ai-for-data-science/
🔍 Fetching and extracting content 

## Dùng Together AI và DeepSeek-R1-Distill-Llama-70B

In [25]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import asyncio
import aiohttp
from tavily import TavilyClient
from together import Together
from openai import OpenAI
from google.colab import userdata

# =======================
# Configuration Constants
# =======================
TOGETHER_API_KEY = userdata.get("TOGETHER_API_KEY")
TAVILY_API_KEY = userdata.get("TAVILY_API_KEY")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")  # OpenAI API Key

DEFAULT_MODEL_TOGETHER = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
FINAL_REPORT_MODEL_OPENAI = "o1-mini"  # OpenAI model hỗ trợ output dài

MAX_LINKS_PER_ITERATION = 10
MAX_TOKENS_INPUT = 4000

# Khởi tạo API client
together_client = Together(api_key=TOGETHER_API_KEY)
openai_client = OpenAI(api_key=OPENAI_API_KEY)
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# ============================
# Asynchronous Helper Functions
# ============================

async def call_together_ai_async(messages, model=DEFAULT_MODEL_TOGETHER):
    """Gửi yêu cầu đến Together AI."""
    try:
        response = await asyncio.to_thread(
            together_client.chat.completions.create,
            messages=messages,
            model=model,
            temperature=0.7,
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ Together AI Error: {e}")
        return None

async def call_openai_async(messages, model=FINAL_REPORT_MODEL_OPENAI):
    """Gửi yêu cầu đến OpenAI API để tạo báo cáo dài."""
    try:
        response = await asyncio.to_thread(
            openai_client.chat.completions.create,
            model=model,
            messages=messages,
            # temperature=0.7, # o1 do not support this param
            max_completion_tokens=4000  # Hỗ trợ output dài
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ OpenAI Error: {e}")
        return None

async def generate_search_queries_async(session, user_query):
    """Tạo danh sách truy vấn tìm kiếm dựa trên chủ đề của người dùng."""
    prompt = (
        "You are an expert research assistant. Given the user's query, generate up to four distinct, "
        "precise search queries that would help gather comprehensive information on the topic. "
        "Return only a Python list of strings, for example: ['query1', 'query2', 'query3']."
    )

    messages = [
        {"role": "system", "content": "You are a helpful and precise research assistant."},
        {"role": "user", "content": f"User Query: {user_query}\n\n{prompt}"}
    ]

    response = await call_together_ai_async(messages)

    if response:
        response_cleaned = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()
        try:
            if response_cleaned.startswith("[") and response_cleaned.endswith("]"):
                search_queries = eval(response_cleaned)
                if isinstance(search_queries, list) and all(isinstance(q, str) for q in search_queries):
                    return search_queries
        except Exception as e:
            print(f"⚠️ Error parsing search queries: {e}\nResponse: {response_cleaned}")

    print("⚠️ Using default search queries due to parsing error.")
    return [
        f"{user_query} overview",
        f"{user_query} latest developments",
        f"{user_query} applications",
        f"{user_query} comparisons"
    ]

async def perform_search_async(session, query):
    """Thực hiện tìm kiếm trên Tavily và trả về danh sách URL hợp lệ."""
    try:
        response = await asyncio.to_thread(
            tavily_client.search,
            query=query,
            max_results=10,
            include_raw_content=True,
            search_depth="advanced"
        )
        links = [item.get("url") for item in response.get("results", []) if "url" in item]
        valid_links = [url for url in links if "youtube.com" not in url and "deepseek.com" not in url]
        return valid_links
    except Exception as e:
        print("❌ Error performing Tavily search:", e)
        return []

async def extract_webpage_text_async(url):
    """Trích xuất nội dung từ một trang web."""
    try:
        response = await asyncio.to_thread(tavily_client.extract, urls=[url], include_images=False)
        if "results" in response and response["results"]:
            return response["results"][0].get("raw_content", "")[:MAX_TOKENS_INPUT]
        print(f"⚠️ No extraction result for URL: {url}")
        return ""
    except Exception as e:
        print(f"❌ Error extracting webpage text with Tavily for {url}: {e}")
        return ""

async def process_link(session, link, user_query, search_query):
    """Xử lý từng link: trích xuất nội dung."""
    print(f"🔍 Fetching and extracting content from: {link}")
    page_text = await extract_webpage_text_async(link)
    return page_text if page_text else None

async def generate_final_report_async(session, user_query, all_contexts):
    """Tạo báo cáo cuối cùng dựa trên nội dung trích xuất, sử dụng OpenAI API để có độ dài cao hơn."""
    if not all_contexts:
        return f"⚠️ Limited information found for: {user_query}."

    context_combined = "\n".join(all_contexts)
    prompt = (
        "You are an AI research assistant. Based on the gathered contexts and original query, "
        "write a **detailed and structured** long-form report. "
        "Ensure that the content is well-organized and provides deep insights."
    )
    messages = [
        {"role": "assistant", "content": "You are a skilled long-form report writer."},
        {"role": "user", "content": f"User Query: {user_query}\n\nExtracted Content:\n{context_combined}\n\n{prompt}"}
    ]
    report = await call_openai_async(messages)  # ✅ Chuyển qua OpenAI model
    return report if report else "⚠️ No significant data available."

# =========================
# Main Asynchronous Routine
# =========================
async def async_main():
    user_query = input("Enter your research query/topic: ").strip()
    iteration_limit = 2

    aggregated_contexts = []

    async with aiohttp.ClientSession() as session:
        all_search_queries = await generate_search_queries_async(session, user_query)
        if not all_search_queries:
            print("❌ No search queries generated. Exiting.")
            return

        for iteration in range(iteration_limit):
            print(f"\n=== Iteration {iteration + 1} ===")
            search_results = await asyncio.gather(*[perform_search_async(session, query) for query in all_search_queries])

            unique_links = set()
            for links in search_results:
                unique_links.update(links)

            limited_links = list(unique_links)[:MAX_LINKS_PER_ITERATION]
            link_results = await asyncio.gather(*[process_link(session, link, user_query, all_search_queries[0]) for link in limited_links])

            aggregated_contexts.extend(filter(None, link_results))

        print("\nGenerating final report...")
        final_report = await generate_final_report_async(session, user_query, aggregated_contexts)
        print("\n==== FINAL REPORT ====\n", final_report)

def main():
    asyncio.run(async_main())

if __name__ == "__main__":
    main()


## phiên bản Version 2 cải tiến hơn!

In [48]:
import os
import re
import asyncio
import aiohttp
from tavily import TavilyClient
from together import Together
from openai import OpenAI
from google.colab import userdata

# =======================
# Configuration Constants
# =======================
TOGETHER_API_KEY = userdata.get("TOGETHER_API_KEY")
TAVILY_API_KEY = userdata.get("TAVILY_API_KEY")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

DEFAULT_MODEL_TOGETHER = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
FINAL_REPORT_MODEL_OPENAI = "o1-mini"  # Supports up to ~65k tokens for completion

# Lựa chọn chunk size & limit
MAX_LINKS_PER_ITERATION = 10
MAX_TOKENS_INPUT = 40000  # Tăng/giảm tùy ý, 40k an toàn
CHUNK_SIZE = 24000        # Mỗi chunk < 24k chars
SUMMARIZE_COMPLETION_TOKENS = 2000  # Summarize chunk
FINAL_COMPLETION_TOKENS = 64000     # <= 65536 limit

together_client = Together(api_key=TOGETHER_API_KEY)
openai_client = OpenAI(api_key=OPENAI_API_KEY)
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# ============================
# Asynchronous Helper Functions
# ============================

async def call_together_ai_async(messages, model=DEFAULT_MODEL_TOGETHER):
    """Gọi Together AI để tạo search queries."""
    try:
        response = await asyncio.to_thread(
            together_client.chat.completions.create,
            messages=messages,
            model=model,
            temperature=0.7,
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("❌ Together AI Error:", e)
        return None

async def call_openai_async(messages, max_comp_tokens, model=FINAL_REPORT_MODEL_OPENAI):
    """
    Gửi yêu cầu đến OpenAI, tùy max_comp_tokens <= 65536.
    """
    try:
        response = await asyncio.to_thread(
            openai_client.chat.completions.create,
            model=model,
            messages=messages,
            max_completion_tokens=max_comp_tokens
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("❌ OpenAI Error:", e)
        return None

# =============== Chunk Summarization =============

async def chunk_text(text, chunk_size=CHUNK_SIZE):
    """
    Chia text thành nhiều chunk < chunk_size.
    """
    for i in range(0, len(text), chunk_size):
        yield text[i: i+chunk_size]

async def summarize_chunk(session, chunk_text):
    """
    Tóm tắt 1 chunk, dùng OpenAI với SUMMARIZE_COMPLETION_TOKENS.
    """
    prompt = (
        "You are an AI summarizer. Summarize the text below into a concise overview, "
        "preserving key details but significantly reducing length.\n\nText:\n"
    )
    messages = [
        {"role": "assistant", "content": "You are a summarizer tool."},
        {"role": "user", "content": prompt + chunk_text}
    ]
    summary = await call_openai_async(messages, max_comp_tokens=SUMMARIZE_COMPLETION_TOKENS)
    return summary if summary else ""

# =============== Generate Search Queries =============
async def generate_search_queries_async(session, user_query):
    """
    Dùng Together AI để tạo query (tối đa 4).
    """
    prompt = (
        "You are an expert research assistant. Given the user's query, generate up to four distinct, "
        "precise search queries that would help gather comprehensive information on the topic. "
        "Return only a Python list of strings, for example: ['query1', 'query2', 'query3']."
    )
    messages = [
        {"role": "system", "content": "You are a helpful and precise research assistant."},
        {"role": "user", "content": f"User Query: {user_query}\n\n{prompt}"}
    ]
    resp = await call_together_ai_async(messages)
    if not resp:
        print("⚠️ No response from Together AI, fallback queries.")
        return [
            f"{user_query} overview",
            f"{user_query} in-depth analysis",
            f"{user_query} use cases",
            f"{user_query} latest developments"
        ]
    # clean <think>...
    resp_clean = re.sub(r"<think>.*?</think>", "", resp, flags=re.DOTALL).strip()
    try:
        if resp_clean.startswith("[") and resp_clean.endswith("]"):
            sq = eval(resp_clean)
            if isinstance(sq, list) and all(isinstance(q, str) for q in sq):
                return sq
    except Exception as e:
        print("⚠️ Error parsing search queries:", e, "\nResponse:", resp_clean)

    # fallback
    return [
        f"{user_query} overview",
        f"{user_query} in-depth analysis",
        f"{user_query} use cases",
        f"{user_query} latest developments"
    ]

# =============== Perform Search ===============
async def perform_search_async(session, query):
    """Tavily search, lọc youtube & deepseek."""
    try:
        resp = await asyncio.to_thread(
            tavily_client.search,
            query=query,
            max_results=10,
            include_raw_content=True,
            search_depth="advanced"
        )
        if not resp:
            return []
        links = [x.get("url") for x in resp.get("results", []) if "url" in x]
        valid_links = [u for u in links if "youtube.com" not in u.lower() and "deepseek.com" not in u.lower()]
        return valid_links
    except Exception as e:
        print("❌ Error performing Tavily search:", e)
        return []

# =============== Extract Webpage Text ===============
async def extract_webpage_text_async(url):
    """
    Trích xuất nội dung từ một trang web bằng Tavily.
    In ra độ dài ban đầu (raw_length) trước khi cắt còn MAX_TOKENS_INPUT.
    """
    try:
        response = await asyncio.to_thread(
            tavily_client.extract,
            urls=[url],
            include_images=False
        )
        if "results" in response and response["results"]:
            raw_text = response["results"][0].get("raw_content", "")
            raw_length = len(raw_text)
            print(f"Raw length for {url}: {raw_length} chars (before cutting)")

            # Cắt còn MAX_TOKENS_INPUT để tránh vượt context
            trimmed_text = raw_text[:MAX_TOKENS_INPUT]
            print(f"Trimmed length for {url}: {len(trimmed_text)} chars (after cutting)\n")
            return trimmed_text
        else:
            print(f"⚠️ No extraction result for URL: {url}")
            return ""
    except Exception as e:
        print(f"❌ Error extracting webpage text with Tavily for {url}: {e}")
        return ""


async def process_link(session, link, user_query, search_query):
    """Lấy text 1 link."""
    print(f"🔍 Fetching and extracting content from: {link}")
    text = await extract_webpage_text_async(link)
    return text if text else None

# =============== Generate Final Report ===============
async def generate_final_report_async(session, user_query, all_contexts):
    """
    Tạo báo cáo cuối cùng, chunk Summaries => final.
    """
    if not all_contexts:
        return f"⚠️ Limited information found for: {user_query}."

    # Gộp all context
    combined = "\n".join(all_contexts)
    if len(combined) <= CHUNK_SIZE:
        # Gọi thẳng
        print("✅ Single chunk (no chunk summarization needed).")
        long_prompt = (
            "You are an AI research assistant. Based on the following contexts and the user query, "
            "create a thoroughly detailed, long-form report. Use headings, bullet points, examples, references, etc. "
            f"Output can be up to {FINAL_COMPLETION_TOKENS} tokens if needed."
        )
        messages = [
            {"role": "assistant", "content": "You are a specialized long-form report writer."},
            {"role": "user", "content": f"User Query: {user_query}\n\nContexts:\n{combined}\n\n{long_prompt}"}
        ]
        final_report = await call_openai_async(messages, max_comp_tokens=FINAL_COMPLETION_TOKENS)
        return final_report if final_report else "⚠️ No significant data."

    # ...nếu text > chunk size => chunk summarization
    print("⚠️ Context too large, using chunk summarization...")

    # Summaries
    summaries = []
    async for chunk in chunk_text(combined, CHUNK_SIZE):
        summary = await summarize_chunk(session, chunk)
        summaries.append(summary)

    # Gom summaries
    summaries_combined = "\n\n".join(summaries)
    print(f"✅ Summaries combined length = {len(summaries_combined)} chars")

    # Gọi final
    final_prompt = (
        "You are an AI research assistant. The text below are chunk summaries. "
        "Please combine them into a single cohesive, multi-sectional, long-form report. "
        f"You may produce up to {FINAL_COMPLETION_TOKENS} tokens. Include references, headings, examples, etc."
    )
    messages = [
        {"role": "assistant", "content": "You are a specialized long-form report writer with no explicit token limit."},
        {
            "role": "user",
            "content": f"User Query: {user_query}\n\nSummaries:\n{summaries_combined}\n\n{final_prompt}"
        }
    ]
    final_report = await call_openai_async(messages, max_comp_tokens=FINAL_COMPLETION_TOKENS)
    return final_report if final_report else "⚠️ No final data."

# =========================
# Main Asynchronous Routine
# =========================
async def async_main():
    user_query = input("Enter your research query/topic: ").strip()
    iteration_input = input("Enter max iterations (default=3): ").strip()
    iteration_limit = int(iteration_input) if iteration_input.isdigit() else 3

    aggregated_contexts = []
    processed_links = set()

    async with aiohttp.ClientSession() as session:
        all_search_queries = await generate_search_queries_async(session, user_query)
        if not all_search_queries:
            print("❌ No search queries generated. Exiting.")
            return

        for iteration in range(iteration_limit):
            print(f"\n=== Iteration {iteration + 1} ===")
            search_results = await asyncio.gather(
                *[perform_search_async(session, q) for q in all_search_queries]
            )

            unique_links = set()
            for links in search_results:
                unique_links.update(links)

            # Loại bỏ link trùng lặp đã xử lý
            new_links = unique_links - processed_links
            if not new_links:
                print("⚠️ No new links found this iteration. Possibly no new queries.")
                break
            processed_links.update(new_links)

            # Giới hạn link
            limited_links = list(new_links)[:MAX_LINKS_PER_ITERATION]

            link_results = await asyncio.gather(*[
                process_link(session, link, user_query, all_search_queries[0])
                for link in limited_links
            ])

            # Thêm nội dung mới vào aggregated_contexts
            aggregated_contexts.extend(filter(None, link_results))

            # ✅ In ra độ dài tổng sau khi fetch iteration này
            combined_now = "\n".join(aggregated_contexts)
            print(f"Current combined length: {len(combined_now)} chars")

        print("\nGenerating final report...")
        final_report = await generate_final_report_async(session, user_query, aggregated_contexts)
        print("\n==== FINAL REPORT ====\n", final_report)


def main():
    asyncio.run(async_main())

if __name__ == "__main__":
    main()


Enter your research query/topic: Deep research feature from OpenAI
Enter max iterations (default=3): 
⚠️ Error parsing search queries: unterminated string literal (detected at line 1) (<string>, line 1) 
Response: ['How does OpenAI's deep research feature work?', 'Applications of OpenAI deep research feature', 'Technical specifications of OpenAI deep research API', 'Future developments in OpenAI deep research capabilities']

=== Iteration 1 ===
🔍 Fetching and extracting content from: https://community.openai.com/t/introduction-to-deep-research-from-openai-livestream/1110988
🔍 Fetching and extracting content from: https://www.news9live.com/technology/artificial-intelligence/openai-chatgpt-research-tool-explained-2812331
🔍 Fetching and extracting content from: https://www.cbsnews.com/video/openai-unveils-new-deep-research-tool-for-chatgpt/
🔍 Fetching and extracting content from: https://economictimes.indiatimes.com/news/international/global-trends/openai-launches-deep-research-tool-after